In [40]:
import numpy as np
import matplotlib.pyplot as plt
import cirq

In [52]:
p = 0.01
sigma_x = cirq.unitary(cirq.X)
sigma_y = cirq.unitary(cirq.Y)
sigma_z = cirq.unitary(cirq.Z)
sigma_i = cirq.unitary(cirq.I)
hadamard = cirq.unitary(cirq.H)

In [72]:
bit_flip_ops = [np.sqrt(1-p) * sigma_i, np.sqrt(p) * sigma_x]
depolarizing_ops = [np.sqrt(1-p) * sigma_i, np.sqrt(p/3) * sigma_x, np.sqrt(p/3) * sigma_y, np.sqrt(p/3) * sigma_z]
phase_flip_ops = [np.sqrt(1-p) * sigma_i, np.sqrt(p) * sigma_z]
correlated_phase_flip_ops = [np.sqrt(1- 3/2*p) * np.kron(sigma_i, sigma_i), np.sqrt(p/2) * np.kron(sigma_z, sigma_i), np.sqrt(p/2) * np.kron(sigma_i, sigma_z), np.sqrt(p/2) * np.kron(sigma_z, sigma_z)]

In [73]:
def apply_channel(kraus_ops, opr):
    return sum([np.dot(kraus_op, np.dot(opr, np.conj(kraus_op).T)) for kraus_op in kraus_ops])

def kraus_to_affine(kraus_ops):
    t = [np.trace(sigma @ apply_channel(kraus_ops, sigma_i)).real / 2 for sigma in (sigma_x, sigma_y, sigma_z)]
    M = np.zeros((3, 3))
    paulis = (sigma_x, sigma_y, sigma_z)
    for i in range(3):
        for j in range(3):
            M[i, j] = np.trace(paulis[i] @ apply_channel(kraus_ops, paulis[j])).real / 2
    return M, t

def avg_fid_of_channel(kraus_ops):
    M, _ = kraus_to_affine(kraus_ops)
    return (1 + np.trace(M).real / 3) / 2

Average fidelity should be $\frac{1}{3} ( 1 + 2 \langle a^* a \rangle)$

In [74]:
print(avg_fid_of_channel(bit_flip_ops))
kraus_to_affine(bit_flip_ops)


0.9933333333333334


(array([[1.  , 0.  , 0.  ],
        [0.  , 0.98, 0.  ],
        [0.  , 0.  , 0.98]]),
 [0.0, 0.0, 0.0])

In [75]:
print(avg_fid_of_channel(depolarizing_ops))
kraus_to_affine(depolarizing_ops)

0.9933333333333334


(array([[0.98666667, 0.        , 0.        ],
        [0.        , 0.98666667, 0.        ],
        [0.        , 0.        , 0.98666667]]),
 [0.0, 0.0, 0.0])

In [77]:
# print(avg_fid_of_channel(correlated_phase_flip_ops))
# kraus_to_affine(correlated_phase_flip_ops)

In [57]:
1- p + p/3

0.9933333333333333

In [58]:
np.linalg.eigvals(kraus_to_affine(bit_flip_ops)[0])

array([1.  , 0.98, 0.98])

In [59]:
np.linalg.eigvals(kraus_to_affine(depolarizing_ops)[0])

array([0.98666667, 0.98666667, 0.98666667])